In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn import pipeline
import sklearn
import xgboost
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from numpy.random import seed
from numpy.random import normal
import joblib

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.mode.chained_assignment = None

In [2]:
data_dict = {
    'Yes':'../Data/Synthetic_train_data_with_exp.csv',
    'No':'../Data/Synthetic_train_data_without_exp.csv'
}

model_dict = {
    'GB':{
        'Yes':'../model/GB_exp_Yes.sav',
        'No':'../model/GB_exp_No.sav'
    },
    'RF':{
        'Yes':'../model/RF_exp_Yes.sav',
        'No':'../model/RF_exp_No.sav'
    },
    'XGB':{
        'Yes':'../model/XGB_exp_Yes.sav',
        'No':'../model/XGB_exp_No.sav'
    }
}

In [3]:
for key in data_dict:
    if key=='Yes': continue
    print(key)
    data = pd.read_csv(data_dict[key])
    
    for model_type in ['GB', 'RF', 'XGB']:
        model = joblib.load(model_dict[model_type][key])
        
        rv=[]
        pv=[]
        
        if key=='Yes':
            for p in range(1,7):
                #for exp in range(1, 1901):
                X = data[data.P==p].iloc[:, :-1].values
                rv.extend(data[data.P==p]['target'].values)
                pv.extend(model[p-1].predict(X))

        elif key=='No':
            for p in range(1,7):
                X = data[data.P==p].iloc[:, [0,2,3,4,5,6,7,8,9,10]].values
                rv.extend(data[data.P==p]['target'].values)
                pv.extend(model[p-1].predict(X))
                
        print(data.shape[0], len(rv), len(pv))
        
        if len(rv)==data.shape[0] and len(pv)==data.shape[0]:
            rmse = np.sqrt(mean_squared_error(rv,pv))
            mae = np.sqrt(mean_absolute_error(rv,pv))
            print(f"Model={model_type}, useExp={key}, RMSE={rmse:.3f}, MAE={mae:.3f}")
            
del [rv, pv, data, model, X]

No


AttributeError: 'HalfSquaredError' object has no attribute 'get_init_raw_predictions'

## Train error of Gradient boost with exp value

In [6]:
data = pd.read_csv('../Data/Synthetic_train_data_without_exp.csv')
print(data.shape)
data.head()

(2246982, 11)


,P,w1,m1,w2,m2,w3,m3,w4,m4,w5,target
0,1,0.0,0.000000,0.1,0.425324,0.2,0.854600,0.3,1.281640,0.4,1.698087
1,1,0.1,0.425324,0.2,0.854600,0.3,1.281640,0.4,1.698087,0.5,2.124392
2,1,0.2,0.854600,0.3,1.281640,0.4,1.698087,0.5,2.124392,0.6,2.533590
3,1,0.3,1.281640,0.4,1.698087,0.5,2.124392,0.6,2.533590,0.7,2.957967
4,1,0.4,1.698087,0.5,2.124392,0.6,2.533590,0.7,2.957967,0.8,3.359399


In [7]:
model = joblib.load('../model/XGB_exp_No.sav')
model

[Pipeline(steps=[('xgb',
                  XGBRegressor(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eta=0.1,
                               eval_metric=None, feature_types=None, gamma=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimator

In [8]:
rv=[]
pv=[]
for p in range(1,7):
    X = data[data.P==p].iloc[:, [0,2,3,4,5,6,7,8,9,10]].values
    rv.extend(data[data.P==p]['target'].values)
    #print(X.shape)
    pv.extend(model[p-1].predict(X))
    #print(len(pv))
    #break

In [10]:
np.sqrt(mean_squared_error(rv,pv)), np.sqrt(mean_absolute_error(rv,pv))

(2.8675342103362267, 1.2196958866388758)

In [35]:
for p in range(1,7):
    for exp in range(1, 1901):
        X = data[np.logical_and(data.P==p, data.Exp==exp)].iloc[:, :-1].values
        rv = data[np.logical_and(data.P==p, data.Exp==exp)]['target'].values
        print(X.shape)
        pv = model[p-1].predict(X)
        print(len(pv))
        break
    break

AttributeError: 'DataFrame' object has no attribute 'Exp'

In [32]:
!python -V

Python 3.11.4
